In [1]:
! pip install kaggle

In [2]:
! conda install -c conda-forge librosa --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - librosa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astroid-2.5                |   py36h5fab9bb_1         297 KB  conda-forge
    flask-cors-3.0.8           |             py_0          14 KB  conda-forge
    fsspec-0.8.7               |     pyhd8ed1ab_0          72 KB  conda-forge
    lxml-4.6.2                 |   py36h04a5ba7_1         1.5 MB  conda-forge
    pillow-8.1.0               |   py36ha6010c0_2         670 KB  conda-forge
    pip-21.0.1                 |     pyhd8ed1ab_

In [102]:
import sagemaker
import boto3
from sagemaker import get_execution_role


import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.utils import shuffle
from collections import Counter
from tqdm import tqdm
import os
from scipy import signal

In [103]:
sagemaker_session = sagemaker.Session()
sagemaker_session.boto_region_name
role = get_execution_role()

In [104]:
bucket = 'birdsound-identification-dataset'
s3 = boto3.resource('s3')

In [105]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/ec2-user/.kaggle’: File exists


In [106]:
!kaggle competitions download -c birdclef-2021 -f train_metadata.csv
!unzip train_metadata.csv.zip
!rm train_metadata.csv.zip

  0%|                                               | 0.00/1.94M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.94M/1.94M [00:00<00:00, 78.3MB/s]
Archive:  train_metadata.csv.zip
replace train_metadata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [107]:
meta_data_path = 'train_metadata.csv'
meta_data = pd.read_csv(meta_data_path)
meta_data = meta_data[meta_data['rating'] >= 4 ]
counter_classes = pd.DataFrame(sorted(Counter(meta_data['primary_label']).items(), key=lambda x: x[1], reverse=True))
meta_data = meta_data.merge(counter_classes, how='left', left_on='primary_label', right_on=0)

del meta_data[0]
meta_data = meta_data.rename(columns={1:'count_class'})

meta_data = meta_data[meta_data['count_class']>= 200]
meta_data = shuffle(meta_data)
audio_name = meta_data.loc[meta_data['count_class'] >= 200, 'filename'].values
audio_labels = meta_data.loc[meta_data['count_class'] >= 200, 'primary_label'].values

cls_id_map = {name:idx for idx, name in enumerate(np.unique(audio_labels))}

In [153]:
SR = 22050
SIGNAL_LENGTH = 5 
SPEC_SHAPE = (13, 96) 
FMIN = 500
FMAX = SR//2
HOP_LENGHT = int(SIGNAL_LENGTH * SR / (SPEC_SHAPE[1] - 1))
DURATION  = 10

def open_audio(path):
    waves, rate = librosa.load(path, sr=SR, duration=DURATION)
    processed_audios = []
    for i in range(0, len(waves), int(SIGNAL_LENGTH * SR)):
        w = waves[i:i + int(SIGNAL_LENGTH * SR)]
        if len(w) < int(SIGNAL_LENGTH * SR):
            break
        if np.any(np.isnan(w)) == False:
            stft = signal.stft(w, nperseg=SR*5//256)[-1]
            stft = np.abs(stft)
            stft = np.log(stft)
            stft = stft - np.min(stft)
            stft = stft/np.max(stft)
            processed_audios.append(stft)
    return processed_audios

In [159]:
!mkdir audio_files

dir_path = 'audio_files/'
data_path = 'data/'

for audio_file, target in tqdm(zip(audio_name[:10], audio_labels[:10])):
    !kaggle competitions download -c birdclef-2021 -p audio_files/ -f {'train_short_audio/' + target + '/' + audio_file} 

    audio_path = dir_path + str(audio_file)
    
    
    try:
        temp = open_audio(audio_path)
        if not os.path.exists(data_path + target):
            os.makedirs(data_path + target)

        for idx, x in enumerate(temp):
            print(x.shape)
            img = Image.fromarray(x * 255.0).convert("L")
            img_save_path = data_path + target + '/' + audio_file[:-4] + f'_{idx}' '.png'
            img.save(img_save_path)
            s3.Bucket(bucket).upload_file(img_save_path, img_save_path)
            !rm {img_save_path}
        !rm {dir_path + audio_file}

    except:
        continue

mkdir: cannot create directory ‘audio_files’: File exists


0it [00:00, ?it/s]

XC323977.ogg.zip: Skipping, found more recently modified local copy (use --force to force download)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
1it [00:01,  1.06s/it]

  0%|                                                | 0.00/288k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 288k/288k [00:00<00:00, 46.9MB/s]
(216, 514)
(216, 514)


...

8it [00:12,  1.64s/it]

  0%|                                                | 0.00/355k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 355k/355k [00:00<00:00, 49.0MB/s]
(216, 514)
(216, 514)


9it [00:13,  1.69s/it]

  0%|                                                | 0.00/104k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 104k/104k [00:00<00:00, 35.2MB/s]
(216, 514)


10it [00:15,  1.54s/it]


In [157]:
open_audio(audio_path)

[array([[0.25945017, 0.40224954, 0.4476753 , ..., 0.57829994, 0.6254348 ,
         0.7122988 ],
        [0.27175063, 0.38472182, 0.4950133 , ..., 0.6174457 , 0.5876202 ,
         0.71205163],
        [0.28659174, 0.42749974, 0.4858006 , ..., 0.62800777, 0.571841  ,
         0.71327233],
        ...,
        [0.06736848, 0.1915573 , 0.36595547, ..., 0.37171593, 0.5975893 ,
         0.70108885],
        [0.0753105 , 0.18281052, 0.34462327, ..., 0.3408085 , 0.59473515,
         0.7010831 ],
        [0.08076826, 0.11897497, 0.32239306, ..., 0.2961362 , 0.5955281 ,
         0.7010669 ]], dtype=float32)]

In [73]:
!mkdir audio_files

dir_path = 'audio_files/'
data_path = 'data/'

targets = []
data = np.zeros(SR*SIGNAL_LENGTH)
for audio_file, target in tqdm(zip(audio_name[:10], audio_labels[:10])):
    
    !kaggle competitions download -c birdclef-2021 -p audio_files/ -f {'train_short_audio/' + target + '/' + audio_file} 
    audio_path = dir_path + str(audio_file)
    
    try:
        temp = open_audio(audio_path)
        for idx, x in enumerate(temp):
            
            targets = targets + len(temp)*[target]
            data = np.vstack((data, temp))
        
        !rm {dir_path + audio_file}

    except:
        continue

mkdir: cannot create directory ‘audio_files’: File exists


0it [00:00, ?it/s]

  0%|                                                | 0.00/274k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 274k/274k [00:00<00:00, 20.6MB/s]


1it [00:01,  1.72s/it]

  0%|                                                | 0.00/321k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 321k/321k [00:00<00:00, 39.4MB/s]


...

8it [00:12,  1.57s/it]

  0%|                                                | 0.00/110k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 110k/110k [00:00<00:00, 25.4MB/s]


9it [00:14,  1.53s/it]

  0%|                                                | 0.00/162k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 162k/162k [00:00<00:00, 44.6MB/s]


10it [00:15,  1.57s/it]


In [75]:
data = pd.DataFrame(data[1:])
data['label'] = targets

In [76]:
data.to_csv('data.csv', index=False)

s3.Bucket(bucket).upload_file('data.csv', 'data.csv')